# A basic `TensorFlow` installation check

Pre-reqs: If nothing is working right, check this notebook, [../jims_ml_notebook_env.py].

A sanity check call to `tensorflow` to make sure it finds and loads the library. NOTE: this does not do any tensor processing:

In [1]:
import numpy as np
import tensorflow as tf

print(f"A TF computation: {tf.add(1, 5).numpy()}")

hello = tf.constant('Hello, TensorFlow!')
print(f"A TensorFlow string constant: {hello.numpy()}")

A TF computation: 6
A TensorFlow string constant: b'Hello, TensorFlow!'


# Of special `import` for `keras`
Keras notions should be imported from `keras.*`, not from `tensorflow`.

When coding to the conventional API:

In [2]:

# Imports when using the conventional class-centric Keras API

# from keras.api.datasets import mnist
from keras.api.models import Sequential
from keras.api.layers import Input, Dense




When coding to the newer functional Keras API:


Now we load a dataset from the convenient tensorflow_datasets python package.

In [3]:
import importlib
import jkcsoft.ml.tensorflow_utils

importlib.reload(jkcsoft.ml.tensorflow_utils)

from jkcsoft.ml.tensorflow_utils import dump_dataset_info, display_dataset_info

# Validates datasets package install and lists voluminous available datasets
dump_dataset_info()

# Create an array of 5 most popular datasets
most_popular_datasets = ['mnist', 'cifar10', 'imdb_reviews', 'fashion_mnist', 'coco']
print(f"Most popular datasets: {most_popular_datasets}")

for dataset_name in most_popular_datasets:
    display_dataset_info(dataset_name)


2025-03-12 16:44:39.106353: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Total datasets available: 1303
A few available datasets: ['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'ai2dcaption', 'aloha_mobile', 'amazon_us_reviews'] ...
Most popular datasets: ['mnist', 'cifar10', 'imdb_reviews', 'fashion_mnist', 'coco']

Dataset: mnist
Description: The MNIST database of handwritten digits.
Features: FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})
Supervised keys: ('image', 'label')
Splits: {'test': <SplitInfo num_examples=10000, num_shards=1>, 'train': <SplitInfo num_examples=60000, num_shards=1>}
Dataset size: 21.00 MiB bytes
Citation: @article{lecun2010mnist,
  title={MNIST handwritten digit database},
  author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
  journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
  volume={2},
  year={2010}
}

Dataset: cifar10
Description: The CIFAR-10 dataset con

# Setup training and test datums to be used in the next two examples

The common datums are used to compare the use of the two Keras APIs.


In [4]:

import os
from jkcsoft.ml.test_results_db.ml_results_database_orm import ModelTestRun, RunBatch, TestDataset

# define a batch include a common dataset for all tests
run_batch = RunBatch()
nb_name = "jims_ml_tensorflow.ipynb"
dataset = TestDataset(f"adhoc {nb_name}",
                      x_train=np.array([[1.0], [2.0], [3.0], [4.0]]),
                      y_train=np.array([[2.0], [4.0], [6.0], [8.0]]),
                      x_test=np.array([[5.0], [6.0]]),
                      y_test=np.array([[10.0], [12.0]])
                      )

dataset

TestDataset(name='adhoc jims_ml_tensorflow.ipynb', x_train shape=(4, 1), y_train shape=(4, 1), x_test shape=(2, 1), y_test shape=(2, 1))

# A simple _conventional_ Keras pipeline with hard-coded TensorFlow datasets

In [5]:
# Define a model using the conventional Keras class API
model = Sequential([
    Input(shape=(1,)),  # Input layer for single-dimensional input
    Dense(32, activation="relu"),  # kera.src.core.dense.Dense
    Dense(16, activation="sigmoid"),
    Dense(16, activation="softmax"),
    Dense(1, activation="linear")
])

test_common_dense = ModelTestRun(TestDataset(""), model)
test_common_dense.description = "Common Dense Sequence"
test_common_dense.fit_epochs(10)
test_common_dense.backend = "tensorflow"
test_common_dense.frontend = "keras"
test_common_dense.backend_proc = "cpu"

# invoke model compile here
test_common_dense.compile_model(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        "mean_absolute_error",
        "mean_absolute_percentage_error",
        "mean_squared_error",
        "root_mean_squared_error",
        "accuracy",
        "precision",
    ])

# test_common_dense.model.fit()

run_batch.add_test(test_common_dense)

# run all models against the same batch dataset
run_batch.run_all()

print('\nTest accuracy:', run_batch.results[0].eval_results)

AttributeError: 'ModelTestRun' object has no attribute 'set_description'

# A simple pipeline with the Keras _functional_ API

In [ ]:
import keras
from keras import layers

keras.backend.clear_session()

# Keep the input tensor
inputs_fn = keras.Input(shape=(1,))

print(f"inputs shape: {inputs_fn.shape}")

print(f"inputs dtype: {inputs_fn.dtype}")

# pipe input
d1 = layers.Dense(32, activation="relu")
x = d1(inputs_fn)

# pipe thru proc layers
d2 = layers.Dense(16, activation="sigmoid")
x = d2(x)
d3 = layers.Dense(8, activation="softmax")
x = d3(x)

# use final layer to reduce to output
final_layer = layers.Dense(1, activation="linear")
outputs_fn = final_layer(x)

functional_model = keras.Model(inputs=inputs_fn, outputs=outputs_fn, name="functional_model_1")

print(f"functional_model summary: {functional_model.summary()}")

functional_model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        "mean_absolute_error",
        "mean_absolute_percentage_error",
        "mean_squared_error",
        "root_mean_squared_error",
        "accuracy",
        "precision",
        "recall",
        "auc",
        "binary_accuracy"
    ]
)

functional_model.fit(run_batch.x_train, run_batch.y_train, epochs=3, verbose=1)

loss_and_metrics = functional_model.evaluate(run_batch.x_test, run_batch.y_test, verbose=2)

print(f"Test eval results: {loss_and_metrics}")

## A more readable functional pipeline declaration

In [ ]:
from jkcsoft.ml import keras_utils

importlib.reload(keras_utils)

proc_layers = [d1, d2, d3, final_layer]

composite_model = keras_utils.compose_model(inputs_fn, proc_layers, verbose=False)

print(f"composite_model summary: {composite_model.summary()}")

composite_model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        #        "mean_absolute_error",
        "mean_absolute_percentage_error",
        #        "mean_squared_error",
        #        "root_mean_squared_error",
        #        "accuracy",
        #        "precision",
        #        "recall",
        #        "auc",
        #        "binary_accuracy"
    ]
)

composite_model.fit(run_batch.x_train, run_batch.y_train, epochs=3, verbose=1)

loss_and_metrics = composite_model.evaluate(run_batch.x_test, run_batch.y_test, verbose=2)

print(f"Test eval results: {loss_and_metrics}")

## A _functional_ model loaded into our `RunBatch` apparatus

In [ ]:
keras.backend.clear_session()

test_common_dense_fn = ModelTestRun().set_description("Common Dense Sequence").set_model(functional_model)
test_common_dense_fn.set_fit_epochs(10)
test_common_dense_fn.compile_model(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        #        "mean_absolute_error",
        "mean_absolute_percentage_error",
        #        "mean_squared_error",
        #        "root_mean_squared_error",
        #        "accuracy",
        #        "precision",
        #        "recall",
        #        "auc",
        #        "binary_accuracy"
    ]
)

test_common_dense_fn.model.summary()

test_common_dense_fn.model.fit(run_batch.x_train, run_batch.y_train, epochs=10, verbose=1)

run_batch.add_test(test_common_dense_fn)

run_batch.run_all()

for i, result in enumerate(run_batch.results):
    print(f"Test {i} eval results: {result.eval_results}")
